In [1]:
from cluster.RayJobManager import RayJobManager

In [2]:
RAY_ADDRESS  = "http://10.254.168.230:8265"
#RAY_ADDRESS  = "local"
ENTRYPOINT   = "python3 cluster/tune_grid.py"
WORKING_DIR  = "/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/"
PIP = {
    "packages": [
        "torch",
        "ray[tune]",
        "git+https://github.com/EmanueleCoradin/TrackSelectorDNN.git@main"
    ],
    "pip_check": False,
    "pip_install_options": ["--verbose"]
}

ray_manager = RayJobManager(ray_address=RAY_ADDRESS)

In [ ]:
job_id = ray_manager.submit_job(
    entrypoint=ENTRYPOINT,
    runtime_env={
        "working_dir": WORKING_DIR, 
        "pip": PIP, 
    }
    
)

ray_manager.wait_until_done(job_id)

2025-11-05 13:59:54,316	INFO dashboard_sdk.py:338 -- Uploading package gcs://_ray_pkg_4ba514d40388c7b4.zip.
2025-11-05 13:59:54,319	INFO packaging.py:588 -- Creating a file package for local module '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/'.


Submitted Ray job: raysubmit_ztTcrXYhQwUsKtzb
Job status: PENDING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING


In [4]:
!pip uninstall -y TrackSelectorDNN

Found existing installation: TrackSelectorDNN 0.1
Uninstalling TrackSelectorDNN-0.1:
  Successfully uninstalled TrackSelectorDNN-0.1


In [3]:
!pip install -e ../TrackSelectorDNN

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///eos/home-i03/e/ecoradin/GitHub/TrackSelectorDNN
  Preparing metadata (setup.py) ... done
  Attempting uninstall: TrackSelectorDNN
    Found existing installation: TrackSelectorDNN 0.1
    Uninstalling TrackSelectorDNN-0.1:
      Successfully uninstalled TrackSelectorDNN-0.1
  DEPRECATION: Legacy editable install of TrackSelectorDNN==0.1 from file:///eos/home-i03/e/ecoradin/GitHub/TrackSelectorDNN (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for TrackSelectorDNN


In [1]:
#Uncomment to run locally
from ray import tune
from TrackSelectorDNN.configs.schema import load_config
from TrackSelectorDNN.tune.trainable import trainable

base_cfg = load_config("base.yaml").model_dump()

search_space = {
    **base_cfg,  # include all validated defaults
    "lr": tune.loguniform(1e-5, 1e-2),
    "weight_decay": tune.loguniform(1e-5, 1e-2),
    "latent_dim": tune.choice([16, 32, 64]),
    "pooling_type": tune.choice(["sum", "mean", "softmax"]),
    "netA_hidden_layers": tune.choice([1, 2, 3, 4, 5]),
    "netB_hidden_layers": tune.choice([1, 2, 3, 4, 5]),
}


tuner = tune.Tuner(
    tune.with_resources(
        trainable,
        {"cpu": 1, "gpu": 0}  # adjust CPU/GPU numbers as needed
    ),
    param_space=search_space,
    tune_config=tune.TuneConfig(metric="val_loss", mode="min", num_samples=4),
)

results = tuner.fit()

2025-11-05 11:07:51,906	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2025-11-05 11:07:51,948	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/eos/user/e/ecoradin/ray_results/trainable_2025-11-05_11-06-52' in 0.0403s.
2025-11-05 11:08:01,974	INFO tune.py:1041 -- Total run time: 49.68 seconds (39.15 seconds for the tuning loop).
2025-11-05 11:08:01,975	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="/eos/user/e/ecoradin/ray_results/trainable_2025-11-05_11-06-52", trainable=...)


In [15]:
best_result = results.get_best_result(metric="val_loss", mode="min")

print("Best config found:")
print(best_result.config)

best_ckpt_dir = best_result.checkpoint
print("Best checkpoint path:", best_ckpt_dir)

Best config found:
{'hit_input_dim': 10, 'track_feat_dim': 16, 'latent_dim': 16, 'pooling_type': 'sum', 'netA_hidden_dim': 32, 'netA_hidden_layers': 1, 'netA_batchnorm': True, 'netA_activation': 'silu', 'netB_hidden_dim': 64, 'netB_hidden_layers': 1, 'netB_batchnorm': True, 'netB_activation': 'silu', 'lr': 0.0002934404584581068, 'epochs': 10, 'batch_size': 64, 'dataset_type': 'dummy', 'dummy_load_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/dummy_dataset.pt', 'train_path': 'data/processed/tracks_train.pt', 'n_tracks': 100000, 'max_hits': 15, 'val_fraction': 0.2, 'weight_decay': 0.0002969065740084239}
Best checkpoint path: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-05_09-16-11/trainable_123a7_00000_0_latent_dim=16,lr=0.0003,netA_hidden_layers=1,netB_hidden_layers=1,pooling_type=sum,weight_decay=0.0003_2025-11-05_09-16-11/checkpoint_000009)


In [1]:
import json
import os
import torch
from TrackSelectorDNN.models.track_classifier import TrackClassifier

run_root = "/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs"
best_val = float("inf")
best_ckpt_path = None
best_config = None

for trial_name in os.listdir(run_root):
    trial_dir = os.path.join(run_root, trial_name)
    metrics_file = os.path.join(trial_dir, "best_metrics.json")
    ckpt_file = os.path.join(trial_dir, "best_model.pt")

    if os.path.exists(metrics_file):
        with open(metrics_file, "r") as f:
            metrics = json.load(f)
        if metrics["val_loss"] < best_val:
            best_val = metrics["val_loss"]
            best_ckpt_path = ckpt_file
            # optionally save config
            config_file = os.path.join(trial_dir, "config.yaml")
            best_config = config_file if os.path.exists(config_file) else None

print("Best checkpoint:", best_ckpt_path)
print("Best val_loss:", best_val)
print("Best config file:", best_config)

Best checkpoint: /eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs/2025-11-04_15-48-29/best_model.pt
Best val_loss: 0.010471555917942896
Best config file: /eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs/2025-11-04_15-48-29/config.yaml


In [5]:
import yaml

# Open and load YAML
with open(best_config, "r") as f:
    best_config = yaml.safe_load(f)

In [9]:
model = TrackClassifier(hit_input_dim = best_config["hit_input_dim"],
                        track_feat_dim = best_config["track_feat_dim"],
                        latent_dim = best_config["latent_dim"],
                        pooling_type = best_config["pooling_type"],
                        # --- NetA parameters ---
                        netA_hidden_dim = best_config["netA_hidden_dim"],
                        netA_hidden_layers = best_config["netA_hidden_layers"],
                        netA_batchnorm = best_config["netA_batchnorm"],
                        netA_activation = best_config["netA_activation"],
                        # --- NetB parameters ---
                        netB_hidden_dim = best_config["netB_hidden_dim"],
                        netB_hidden_layers = best_config["netB_hidden_layers"],
                        netB_batchnorm = best_config["netB_batchnorm"],
                        netB_activation = best_config["netB_activation"])


model.load_state_dict(torch.load('/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs/2025-11-04_15-48-29/best_checkpoint/model.pt', map_location="cpu"))
model.eval()

TrackClassifier(
  (pool): SumPooling()
  (netA): NetA(
    (mlp): Sequential(
      (0): Linear(in_features=10, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU()
      (3): Linear(in_features=32, out_features=32, bias=True)
      (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): SiLU()
      (6): Linear(in_features=32, out_features=32, bias=True)
      (7): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): SiLU()
      (9): Linear(in_features=32, out_features=64, bias=True)
    )
  )
  (netB): NetB(
    (mlp): Sequential(
      (0): Linear(in_features=80, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU()
      (3): Linear(in_features=64, out_features=1, bias=True)
    )
  )
)

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

def plot_run_losses(run_dir):
    log_file = os.path.join(run_dir, "metrics.csv")  # or whatever CSVLogger writes
    if not os.path.exists(log_file):
        raise FileNotFoundError(f"No metrics.csv found in {run_dir}")

    df = pd.read_csv(log_file)

    # Assumes columns: epoch, train_loss, val_loss
    epochs = df["epoch"]
    train_loss = df["train_loss"]
    val_loss = df["val_loss"]

    plt.figure(figsize=(8,5))
    plt.plot(epochs, train_loss, label="Train Loss")
    plt.plot(epochs, val_loss, label="Validation Loss")

    plt.title("Training & Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
plot_run_losses("/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/demo_runs/2025-11-04_15-48-29/")